In [ ]:
import numpy as np
import xarray as xr
import os,errno
import sys
%matplotlib inline

dir='/Volumes/Extreme Pro/'

In [ ]:
yrs=np.arange(2023,2024,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    #ds1=xr.open_dataset(dir+'era_5_ivtu_'+str(year)+'.nc')
    ds2=xr.open_dataset(dir+'era_5_iwv_'+str(year)+'.nc')
    #ds=xr.open_dataset('/Users/ahenny/Downloads/adaptor.mars.internal-1710085183.8506913-30542-1-cedb78e6-3193-4fc4-a935-e5b9ac06cca8.nc')

    lons_west=[x for x in ds.longitude.values if x<180]
    lons_east=[x for x in ds.longitude.values if x>=180]
    if year%4==0:
        ds3=xr.open_dataset(dir+'merra2.ar.labels.model.variable.'+str(1980)+'.nc')
        ds3['time']=ds.time.values
    else:
        ds3=xr.open_dataset(dir+'merra2.ar.labels.model.variable.'+str(1981)+'.nc')
        ds3['time']=ds.time.values
    
    for j in range(ds.time.size):
        date=ds.time.values[j]
        print(date)
        ivtu_west=ds['p71.162'].sel(time=date,longitude=lons_west)
        ivtu_east=ds['p71.162'].sel(time=date,longitude=lons_east)
        ivtu_east['longitude']=[x-360 for x in lons_east]
        ivtu=xr.concat([ivtu_east,ivtu_west],dim='longitude')
        ivtv_west=ds['p72.162'].sel(time=date,longitude=lons_west)
        ivtv_east=ds['p72.162'].sel(time=date,longitude=lons_east)
        ivtv_east['longitude']=[x-360 for x in lons_east]
        ivtv=xr.concat([ivtv_east,ivtv_west],dim='longitude')
    
        merra2=ds3['ar_labeled'][j,:,:]
        
        #ivtu=ivtu.interp(latitude=merra2.lat.values,longitude=merra2.lon.values)
        #ivtv=ivtv.interp(latitude=merra2.lat.values,longitude=merra2.lon.values)
        #ivt=np.sqrt(ivtu**2+ivtv**2)
        
        if j==0:
            #ivt_concat=ivt
            ivt_concat=ivtv
        else:
            #ivt_concat=xr.concat([ivt_concat,ivt],dim='time')
            ivt_concat=xr.concat([ivt_concat,ivtv],dim='time')
    
    print(ivt_concat)
    
    dk=xr.Dataset()
    dk['ivt']=(('time','lat','lon'),ivt_concat.values)
    dk.coords['time']=ivt_concat.time
    dk.coords['lat']=ivt_concat.latitude.values
    dk.coords['lon']=ivt_concat.longitude.values
    
    dk['ivt'].attrs["description"]='IVT v component'
    dk['ivt'].attrs["units"]='kg*m-1*s-1'
    
    try:
        os.remove(dir+'era_5_ivtv_'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir+'era_5_ivtv_'+str(year)+'.nc',mode='w',format='NETCDF4')

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
ds=xr.open_dataset(dir+'era_5_mfcinterp_'+str(2023)+'.nc')
print(ds)
test=ds['mfc'][100,:,:]*1e5
fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(test.lon,test.lat,test,vmin=-20,vmax=20,cmap=plt.cm.Greys)
ax.coastlines(resolution='10m')
plt.show()


ds=xr.open_dataset(dir+'era_5_mfc_'+str(2023)+'.nc')
test=ds['p84.162'][100,:,:]*1e5
fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(test.longitude,test.latitude,test,vmin=-20,vmax=20,cmap=plt.cm.Greys)
ax.coastlines(resolution='10m')
plt.show()

In [ ]:
dir2='/Volumes/Extreme Pro/'
yrs=np.arange(2023,2024,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds1=xr.open_dataset(dir2+'era_5_mfc_'+str(year)+'.nc')

    lons_west=[x for x in ds1.longitude.values if x<180]
    lons_east=[x for x in ds1.longitude.values if x>=180]
    if year%4==0:
        ds3=xr.open_dataset(dir+'merra2.ar.labels.model.variable.'+str(1980)+'.nc')
        ds3['time']=ds1.time.values
    else:
        ds3=xr.open_dataset(dir+'merra2.ar.labels.model.variable.'+str(1981)+'.nc')
        ds3['time']=ds1.time.values
    
    for j in range(ds1.time.size):
        date=ds1.time.values[j]
        print(date)
        
        if 1==1:
            field_west=ds1['p84.162'].sel(time=date,longitude=lons_west)
            field_east=ds1['p84.162'].sel(time=date,longitude=lons_east)
            field_east['longitude']=[x-360 for x in lons_east]
            field=xr.concat([field_east,field_west],dim='longitude')
        else:
            field=ds1['p84.162'].sel(time=date)
    
        merra2=ds3['ar_labeled'][j,:,:]
        
        field=field.interp(latitude=merra2.lat.values,longitude=merra2.lon.values)
        
        if j==0:
            field_concat=field
        else:
            field_concat=xr.concat([field_concat,field],dim='time')
    
    dk=xr.Dataset()
    dk['mfc']=(('time','lat','lon'),field_concat.values)
    dk.coords['time']=field_concat.time
    dk.coords['lat']=field_concat.latitude.values
    dk.coords['lon']=field_concat.longitude.values
    
    dk['mfc'].attrs["description"]='Vertically-integrated moisture flux convergence'
    dk['mfc'].attrs["units"]='kg*m-2*s-1'
    
    try:
        os.remove(dir+'era_5_mfcinterp_'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir+'era_5_mfcinterp_'+str(year)+'.nc',mode='w',format='NETCDF4')

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
ds=xr.open_dataset(dir+'era_5_v850_'+str(1950)+'.nc')
test=ds['v'][100,:,:]
#test=test.interp(lat=merra2.lat.values,lon=merra2.lon.values)
print(test.values)
fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(test.longitude,test.latitude,test,vmin=-20,vmax=20,cmap=plt.cm.seismic)
ax.coastlines(resolution='10m')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
ds=xr.open_dataset(dir+'era_5_v850interp_'+str(1957)+'.nc')
test=ds['v'][100,:,:]
#test=test.interp(lat=merra2.lat.values,lon=merra2.lon.values)
print(test.values)
fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(test.lon,test.lat,test,vmin=-20,vmax=20,cmap=plt.cm.seismic)
ax.coastlines(resolution='10m')
plt.show()